# Exit FTW
por defecto, los procesos que funcionan correctamente dan como estatus de salida, un 0, y los que no, cualquier valor diferente de 0. veamos un par de casos

In [3]:
rm -r test/
bash sources/bash/goodCD.sh
echo "El proceso termino en: $?"

sources/bash/goodCD.sh: line 9: cd: test2: No such file or directory
El proceso termino en: 1


In [2]:
bash sources/bash/badCD.sh
echo "El proceso termino en: $?"

mkdir: cannot create directory ‘test’: File exists
sources/bash/badCD.sh: line 10: cd: test2: No such file or directory
El proceso termino en: 0


con el comando exit (que por defecto regresa un 1) informamos al siguiente que esto fallo, ¿como podemos usarlo?

In [4]:
cat sources/bash/exit.sh

#!/bin/env bash

bash "$(dirname "$0")/goodCD.sh"
#echo "debug print $?"
if [ $? -eq 0 ] 
then
  echo "todo funciono correctamente"
else
  echo "algo fallo"
fi


en un if checamos la salida, y en base a ello ejecutamos

In [5]:
bash -x sources/bash/exit.sh

++ dirname sources/bash/exit.sh
+ bash sources/bash/goodCD.sh
mkdir: cannot create directory ‘test’: File exists
sources/bash/goodCD.sh: line 9: cd: test2: No such file or directory
+ '[' 1 -eq 0 ']'
+ echo 'algo fallo'
algo fallo


se comporto como creemos, entonces shellcheck no deberia de dar errores:

In [6]:
shellcheck sources/bash/exit.sh


In sources/bash/exit.sh line 5:
if [ $? -eq 0 ] 
     ^-- SC2181: Check exit code directly with e.g. 'if mycmd;', not indirectly with $?.



: 1

¿Pero que paso?
quitemos el comentario

In [7]:
cat sources/bash/exitFail.sh
bash -x sources/bash/exitFail.sh

#!/bin/env bash

bash "$(dirname "$0")/goodCD.sh"
echo "debug print $?"
if [ $? -eq 0 ] 
then
  echo "todo funciono correctamente"
else
  echo "algo fallo"
fi
++ dirname sources/bash/exitFail.sh
+ bash sources/bash/goodCD.sh
mkdir: cannot create directory ‘test’: File exists
sources/bash/goodCD.sh: line 9: cd: test2: No such file or directory
+ echo 'debug print 1'
debug print 1
+ '[' 0 -eq 0 ']'
+ echo 'todo funciono correctamente'
todo funciono correctamente


Siempre existe la posibilidad de terminar metiendo codigo muerto dentro de lineas que funcionan, ¿una declaracion de una variable que usaras dentro del loop? ¿un echo de debug?, escoge tu fallo.
veamos la solucion reocmendada

In [8]:
cat  sources/bash/exitFixed.sh
bash -x sources/bash/exitFixed.sh
shellcheck sources/bash/exitFixed.sh

#!/bin/env bash

if bash "$(dirname "$0")/goodCD.sh"  
then
  echo "todo funciono correctamente"
else
  echo "algo fallo"
fi
++ dirname sources/bash/exitFixed.sh
+ bash sources/bash/goodCD.sh
mkdir: cannot create directory ‘test’: File exists
sources/bash/goodCD.sh: line 9: cd: test2: No such file or directory
+ echo 'algo fallo'
algo fallo
